### We fake the CI-MS as input in MSfinder and predict formula

In [3]:
import pandas as pd
import os
import re
def adduct_clean(string):
    if string == '[M + H]+':
        return '[M+H]+'
    elif string == '[M]+':
        return '[M]+'
    elif string == '[M - H2 + H]+':
        return '[M-H]+'
    else:
        return 'others'
    
def formula2dict(formula):
    lst = re.findall(r'[A-Z][a-z]*|\d+', re.sub('[A-Z][a-z]*(?![\da-z])', r'\g<0>1', formula))
    res_dct = {lst[i]: int(lst[i + 1]) for i in range(0, len(lst), 2)}
    return res_dct
def formula2dictH(formula):
    lst = re.findall(r'[A-Z][a-z]*|\d+', re.sub('[A-Z][a-z]*(?![\da-z])', r'\g<0>1', formula))
    res_dct = {lst[i]: int(lst[i + 1]) for i in range(0, len(lst), 2)}
    res_dct['H'] = res_dct['H']+1
    return res_dct
def loss(inputdict):
    inputdict['H'] = inputdict['H'] - 3
    try:
        inputdict['C'] = inputdict['C'] - 1
    except:
        pass
    return inputdict
def calculate_mass(formula: str):
    #from yuanyue
    
    mol_mass = 0.
    try:
        all_atom_nums = re.findall('([A-Z][a-z]*)([0-9]*)', formula)
        for atom_num in all_atom_nums:
            n = atom_num[1]
            if n == '':
                mol_mass += atom_mass[atom_num[0]]
            else:
                mol_mass += int(n) * atom_mass[atom_num[0]]
    except KeyError as e:
        print("Atom {} is not known".format(e.args[0]))
    except TypeError:
        print('formula is nan')
    return round(mol_mass,6)

In [18]:
import collections
def addTMS(formula,i):
    dict1 = [formula2dict(formula)]
    tms = formula2dict('C3H8Si1')
    for j in range(i):
        dict1.append(tms)
    counter = collections.Counter()
    for d in dict1: 
        counter.update(d)
    formulanew = ''
    for key,value in counter.items():
        formulanew += key+str(value)
    return formulanew


def addmeox(formula,i):
    dict1 = [formula2dict(formula)]
    tms = formula2dict('CH3N')
    for j in range(i):
        dict1.append(tms)
    counter = collections.Counter()
    for d in dict1: 
        counter.update(d)
    formulanew = ''
    for key,value in counter.items():
        formulanew += key+str(value)
    return formulanew

In [55]:
ch3loss = False
formula = pd.read_csv('./test/filter_all.csv')
csv = 'rank_filter_msfinder.csv'
path = './filter_msfinder/' #_correct_Mass

In [126]:
atom_mass = {'H': 1.00782503223,
             'C': 12,
             'N': 14.00307400443,
             'O': 15.99491461957,
             'F': 18.99840316273,
             'P': 30.97376199842,
             'S': 31.9720711744,
             'Cl': 34.968852682,
             'Ar': 39.9623831237,
             'K': 38.9637064864,
             'Ca': 39.962590863,
             'Si': 27.976926535,
             'Na':22.98976928
             }

In [54]:
formula

,index,name,inchikey,formula,ionization,precursormz,rank
0,0,norvaline,SNDPXSYFESPGGJ-BYPYZUCNSA-N,C8H19N1O2Si1,[M]+,190.12618,1
1,0,norvaline,SNDPXSYFESPGGJ-BYPYZUCNSA-N,C8H19N1O2Si1,[M]+,190.12618,1
2,1,norvaline,SNDPXSYFESPGGJ-BYPYZUCNSA-N,C11H27N1O2Si2,[M]+,262.16580,1
3,1,norvaline,SNDPXSYFESPGGJ-BYPYZUCNSA-N,C11H27N1O2Si2,[M]+,262.16580,1
4,2,glucosamine 1,MSWZFWKMSRAUBD-QZABAPFNSA-N,C18H45N1O5Si4,[M]+,468.24448,no correct formula
...,...,...,...,...,...,...,...
1261,630,(water loss)4-Hydroxymandelonitrile,HOOOPXDSCKBLFG-UHFFFAOYSA-N,C11H13N1O1Si1,[M]+,202.07201,no correct formula
1262,631,4-Hydroxymandelonitrile,HOOOPXDSCKBLFG-UHFFFAOYSA-N,C14H23N1O2Si2,[M]+,293.12494,no correct formula
1263,631,4-Hydroxymandelonitrile,HOOOPXDSCKBLFG-UHFFFAOYSA-N,C14H23N1O2Si2,[M]+,293.12494,no correct formula
1264,632,(water loss)4-Hydroxymandelonitrile,HOOOPXDSCKBLFG-UHFFFAOYSA-N,C11H13N1O1Si1,[M]+,202.07201,no correct formula


### Read SIRIUS result

In [77]:
i = 0
rank_list = []
name = []
adductformula = []
for txt in os.listdir(path):
    
    #directory = '0_annotate_all_1'
    
    if ('Formula'in txt)&('2098' in txt):
        i += 1
        print(txt)
        index = int(re.findall(r'\d+',txt)[0]) #-1266
        name.append(index)
        try:
            predict = pd.read_csv(path+txt, sep='\t',header=10).iloc[:][1:]
            key = formula[formula['index'] == int(index)] # get the key row from forumla
            predict['molecularFormula'] = predict['Formula'].apply(stdformula)

            predict['dict'] = predict['molecularFormula'].apply(formula2dict)

            
            keydict = formula2dict(key['formula'].item())
            if ch3loss:
                keydict = loss(keydict)
            rank = predict.index[predict['dict'] == keydict].to_list()
            if rank:
                rank_list.append(rank[0])
                
            else:
                rank_list.append('no correct formula')   # in sirius result 
                
        except FileNotFoundError:
            rank_list.append('nan') # didn't calculate in sirius gui
            
        except TypeError:
            rank_list.append('no key formula')
        
    

Formula result-0-2098.txt
Formula result-1-2098.txt
Formula result-10-2098.txt
Formula result-100-2098.txt
Formula result-101-2098.txt
Formula result-102-2098.txt
Formula result-103-2098.txt
Formula result-104-2098.txt
Formula result-105-2098.txt
Formula result-106-2098.txt
Formula result-107-2098.txt
Formula result-108-2098.txt
Formula result-109-2098.txt
Formula result-11-2098.txt
Formula result-110-2098.txt
Formula result-111-2098.txt
Formula result-112-2098.txt
Formula result-113-2098.txt
Formula result-114-2098.txt
Formula result-115-2098.txt
Formula result-116-2098.txt
Formula result-117-2098.txt
Formula result-118-2098.txt
Formula result-119-2098.txt
Formula result-12-2098.txt
Formula result-120-2098.txt
Formula result-121-2098.txt
Formula result-122-2098.txt
Formula result-123-2098.txt
Formula result-124-2098.txt
Formula result-125-2098.txt
Formula result-126-2098.txt
Formula result-127-2098.txt
Formula result-128-2098.txt
Formula result-129-2098.txt
Formula result-13-2098.txt


Formula result-364-2098.txt
Formula result-365-2098.txt
Formula result-366-2098.txt
Formula result-367-2098.txt
Formula result-368-2098.txt
Formula result-369-2098.txt
Formula result-37-2098.txt
Formula result-370-2098.txt
Formula result-371-2098.txt
Formula result-372-2098.txt
Formula result-373-2098.txt
Formula result-374-2098.txt
Formula result-375-2098.txt
Formula result-376-2098.txt
Formula result-377-2098.txt
Formula result-378-2098.txt
Formula result-379-2098.txt
Formula result-38-2098.txt
Formula result-380-2098.txt
Formula result-381-2098.txt
Formula result-382-2098.txt
Formula result-383-2098.txt
Formula result-384-2098.txt
Formula result-385-2098.txt
Formula result-386-2098.txt
Formula result-387-2098.txt
Formula result-388-2098.txt
Formula result-389-2098.txt
Formula result-39-2098.txt
Formula result-390-2098.txt
Formula result-391-2098.txt
Formula result-392-2098.txt
Formula result-393-2098.txt
Formula result-394-2098.txt
Formula result-395-2098.txt
Formula result-396-2098

In [33]:
predict['dict'] = predict['molecularFormula'].apply(formula2dict)

In [38]:
predict['molecularFormula']

1    C8H19N1O2Si1
2    C8H19O2Si1N1
3        C9H19NOS
4     C9H19O1S1N1
Name: molecularFormula, dtype: object

In [29]:
f = 'C9H19NOS'
stdformula(f)

'C9H19NOS'

In [21]:
def stdformula(f):
    tmp = f.split('_')
    formula = tmp[0]
    for item in tmp:
        if 'TMS' in item:
            ntms = int(item.replace('TMS',''))
            formula = addTMS(formula,ntms)
        if 'MEOX' in item:
            nmeox = int(item.replace('MEOX',''))
            formula = addmeox(formula,nmeox)
    return formula

In [32]:
predict['molecularFormula']

1    C8H19N1O2Si1
2    C8H19O2Si1N1
3        C9H19NOS
4     C9H19O1S1N1
Name: molecularFormula, dtype: object

In [78]:
df = pd.DataFrame({'rank':rank_list, 'index':name}) #,'adductformula':adductformula

In [49]:
formula = formula.merge(df,on='index',how='left')

In [79]:
df['rank'].value_counts()

no correct formula    522
1                      68
3                      10
2                       6
7                       5
4                       5
6                       2
5                       2
10                      2
8                       2
20                      2
50                      1
49                      1
29                      1
14                      1
Name: rank, dtype: int64

In [65]:
df

,rank,index
0,1,0
1,1,1
2,7,10
3,no correct formula,100
4,no correct formula,101
...,...,...
1261,no correct formula,95
1262,no correct formula,96
1263,no correct formula,97
1264,no correct formula,98
